In [155]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import FormTrainingClient
from azure.storage.blob import BlobServiceClient
import os
import pandas as pd
from dotenv import load_dotenv
from Person import Person
from datetime import datetime

In [156]:
load_dotenv()

True

In [157]:
AZURE_STORAGE_CONNECTION_STRING = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
AZURE_FORM_RECOGNIZER_ENDPOINT = os.getenv('AZURE_FORM_RECOGNIZER_ENDPOINT')
AZURE_FORM_RECOGNIZER_KEY = os.getenv('AZURE_FORM_RECOGNIZER_KEY')

In [158]:
blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
blob_client = blob_service_client.get_blob_client(container='project1', blob='Flight Manifest.csv')

In [159]:
dir_path = os.getcwd()
download_file_path = dir_path+'/Flight Manifest.csv'

with open(download_file_path, "wb") as download_file:
    download_file.write(blob_client.download_blob().readall())

In [160]:
flight_manifest_df = pd.read_csv('Flight Manifest.csv')
flight_manifest_df

,Carrier,Flight No,Origin,Destination,Date,Time,First Name,Last Name,Sex,Class,SeatNo,DateofBirth,Baggage,DoBValidation,PersonValidation,LuggageValidation,NameValidation,BoardingPassValidation
0,UA,128,Miami,New York,30-May-2022,11:15,Cristopher,Coronado,M,E,15A,10-Aug-1993,0,False,False,False,False,False
1,UA,128,Miami,New York,30-May-2022,11:15,Oscar,Bott,M,B,17B,23-May-1980,2,False,False,False,False,False
2,UA,128,Miami,New York,30-May-2022,11:15,Laura,Olsen,F,B,13B,1-Mar-1996,1,False,False,False,False,False
3,UA,128,Miami,New York,30-May-2022,11:15,Esmeralda,Espinoza,F,E,28A,25-Aug-1995,1,False,False,False,False,False
4,UA,128,Miami,New York,30-May-2022,11:15,Spencer,Williams,M,E,31B,22-Nov-1994,2,False,False,False,False,False


In [161]:
form_recognizer_client = FormRecognizerClient(endpoint=AZURE_FORM_RECOGNIZER_ENDPOINT, credential=AzureKeyCredential(AZURE_FORM_RECOGNIZER_KEY))

In [162]:
form_training_client = FormTrainingClient(endpoint=AZURE_FORM_RECOGNIZER_ENDPOINT, credential=AzureKeyCredential(AZURE_FORM_RECOGNIZER_KEY))
saved_model_list = form_training_client.list_custom_models()
trainingDataUrl = os.getenv('PROJECT1_BOARDING_PASSES_DATA_URL')

In [163]:
training_process = form_training_client.begin_training(trainingDataUrl, use_training_labels=True)
custom_model = training_process.result()

In [164]:
form_recognizer_client = FormRecognizerClient(endpoint=AZURE_FORM_RECOGNIZER_ENDPOINT, credential=AzureKeyCredential(AZURE_FORM_RECOGNIZER_KEY))

In [165]:
digital_ids_path = '../material_preparation_step/Digital IDs'
boarding_passes_path = '../material_preparation_step/Boarding passes'
names = ['Cristopher_Coronado', 'Oscar_Bott', 'Laura_Olsen', 'Esmeralda_Espinoza', 'Spencer_Williams']

index = 0
for name in names:
    with open(digital_ids_path+"/Digital_ID_{}.png".format(name), "rb") as digital_id:
        poller = form_recognizer_client.begin_recognize_identity_documents(identity_document=digital_id)
        digital_id_id_documents = poller.result()
    
    person = Person(digital_id_id_documents[0].fields.get("FirstName").value, digital_id_id_documents[0].fields.get("LastName").value, 
        digital_id_id_documents[0].fields.get("DateOfBirth").value)

    with open(boarding_passes_path+"/Boarding_Pass_{}.pdf".format(name), "rb") as boarding_pass:
        custom_test_action = form_recognizer_client.begin_recognize_custom_forms(model_id=custom_model.model_id, form=boarding_pass)
        boarding_pass_id_documents = custom_test_action.result()

    for recognized_content in boarding_pass_id_documents:
        for name, field in recognized_content.fields.items():
            if name == 'Passenger Name':
                passenger_name_from_boarding_pass = field.value
                first_name_from_boarding_pass = passenger_name_from_boarding_pass.split(' ')[0]
                last_name_from_boarding_pass = passenger_name_from_boarding_pass.split(' ')[1]
            elif name == 'Carrier':
                carrier_from_boarding_pass = field.value
            elif name == 'Flight No.':
                flight_no_from_boarding_pass = int(field.value)
            elif name == 'Class':
                class_from_boarding_pass = field.value
            elif name == 'From':
                from_from_boarding_pass = field.value
            elif name == 'To':
                to_from_boarding_pass = field.value
            elif name == 'Seat':
                seat_from_boarding_pass = field.value
            elif name == 'Date':
                fligh_date_from_boarding_pass = datetime.strptime(field.value, '%b %d, %Y').date()
            elif name == 'Boarding Time':
                fligh_time_from_boarding_pass = field.value

    if (first_name_from_boarding_pass == person.first_name and last_name_from_boarding_pass == person.last_name and
        first_name_from_boarding_pass == flight_manifest_df['First Name'][index] and last_name_from_boarding_pass == flight_manifest_df['Last Name'][index]):
        flight_manifest_df.loc[flight_manifest_df['First Name'] == first_name_from_boarding_pass, 'NameValidation'] = True

    if person.date_birth == datetime.strptime(flight_manifest_df['DateofBirth'][index], '%d-%b-%Y').date():
        flight_manifest_df.loc[flight_manifest_df['First Name'] == first_name_from_boarding_pass, 'DoBValidation'] = True

    if (carrier_from_boarding_pass == flight_manifest_df['Carrier'][index] and flight_no_from_boarding_pass == flight_manifest_df['Flight No'][index]
        and from_from_boarding_pass == flight_manifest_df['Origin'][index] and to_from_boarding_pass == flight_manifest_df['Destination'][index]
        and fligh_date_from_boarding_pass == datetime.strptime(flight_manifest_df['Date'][index], '%d-%b-%Y').date() and fligh_time_from_boarding_pass == flight_manifest_df['Time'][index]
        and seat_from_boarding_pass == flight_manifest_df['SeatNo'][index] and class_from_boarding_pass == flight_manifest_df['Class'][index]):
        flight_manifest_df.loc[flight_manifest_df['First Name'] == first_name_from_boarding_pass, 'BoardingPassValidation'] = True

    index+=1

In [166]:
flight_manifest_df

,Carrier,Flight No,Origin,Destination,Date,Time,First Name,Last Name,Sex,Class,SeatNo,DateofBirth,Baggage,DoBValidation,PersonValidation,LuggageValidation,NameValidation,BoardingPassValidation
0,UA,128,Miami,New York,30-May-2022,11:15,Cristopher,Coronado,M,E,15A,10-Aug-1993,0,True,False,False,True,True
1,UA,128,Miami,New York,30-May-2022,11:15,Oscar,Bott,M,B,17B,23-May-1980,2,True,False,False,True,True
2,UA,128,Miami,New York,30-May-2022,11:15,Laura,Olsen,F,B,13B,1-Mar-1996,1,True,False,False,True,True
3,UA,128,Miami,New York,30-May-2022,11:15,Esmeralda,Espinoza,F,E,28A,25-Aug-1995,1,True,False,False,True,True
4,UA,128,Miami,New York,30-May-2022,11:15,Spencer,Williams,M,E,31B,22-Nov-1994,2,True,False,False,True,True


In [167]:
flight_manifest_df.to_excel('Final Flight Manifest.xlsx')